# Waste Classification using Computer Vision

### Importing the required libraries

In [ ]:
from zipfile import ZipFile
import tensorflow as tf

### Extracting the dataset

In [ ]:
f = ZipFile('DATASET.zip', 'r')
f.extractall()

### Creating the data generators 
* ```train_datagen``` - performing real-time data augmentation on the test images.
* ```test_datagen``` - feature scaling the pixel values of train images.

In [ ]:
batch_size = 16

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

### Getting images from directories
* ```train_generator``` - getting images from ```train_datagen```, resizing them and adding binary labels.
* ```validation_generator``` - getting images from ```train_datagen```, resizing them and adding binary labels.

In [ ]:
train_generator = train_datagen.flow_from_directory(
        'DATASET/TRAIN',  
        target_size=(150, 150), 
        batch_size=batch_size,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        'DATASET/TEST',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

## Building the neural network
We are building a **Convolutional Neural Nework (CNN)**. CNNs are used for image classification and recognition because of its high accuracy. CNNs are fully connected feed forward neural networks. CNNs are very effective in reducing the number of parameters without losing on the quality of models.

### Adding the Convoluional Layers
We are using 3 convolutional layers to with a **ReLU** activation and followed by **max-pooling** layers. This is very similar to the architectures that Yann LeCun advocated in the 1990s for image classification.

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(32, (3, 3), input_shape=(150, 150, 3)))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(tf.keras.layers.Conv2D(32, (3, 3)))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(tf.keras.layers.Conv2D(64, (3, 3)))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

### Adding the Dense Layers
The convolutional layers exract the features from the images which are then predicted the the fully connected dense layes. The final layer has a single output with **sigmoid** activation as this is a binary classification.

In [ ]:
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(1))
model.add(tf.keras.layers.Activation('sigmoid'))

### Defining the loss and optimizer function
* **loss function** - Binary Crossentropy
* **optimizer** - RMSprop 

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

### Training the model
* The model is trained for 50 epochs
* The trained model is stored for future use

In [ ]:
model.fit(
        train_generator,
        batch_size=2000,
        steps_per_epoch=125,
        epochs=50)
model.save('waste-classifier.h5')

### Evaluating the model
The model has an accuracy of about **87%**

In [ ]:
model.evaluate(validation_generator,steps=125)